# Twitter We Rate Dogs Project

## Table of Contents

+ 

## Importing Necessary Libaries

In [7]:
import pandas as pd
import numpy as np
import json
import os
import tweepy
import requests
import re
from tweepy import OAuthHandler
from timeit import default_timer as timer

## Gathering Data

#### Downloading Files Programmatically - Image Prediction File

In [8]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
file_name = url.split("/")[-1]

open(file_name, 'wb').write(response.content)
imagePrediction = pd.read_csv(file_name, sep = '\t')

#### Reading from a TSV File

In [9]:
fileName = 'twitter-archive-enhanced.csv'
twitter_archive = pd.read_csv(fileName)

#### Gathering data from Twitter

In [10]:
with open ("tweet_json.txt") as file:
    list1 = []
    count = 0 
    while True:
        count = count + 1
        text = file.readline()
        if not text:
            break
        text_dict = json.loads(text)
        list1.append(text_dict)

In [11]:
tweet_ids = []
favorite_counts = []
retweet_counts = []

for i in list1:
    ids = i['id']
    tweet_ids.append(ids)
    fvc = i['favorite_count']
    favorite_counts.append(fvc)
    rtc = i['retweet_count']
    retweet_counts.append(rtc)
    
json_dict = {'Tweet_ID':tweet_ids, 'Favorite_counts':favorite_counts, 'Retweet_Counts': retweet_counts}
json_df = pd.DataFrame(json_dict)

#### Combining the Datasets

In [22]:
combinedData = twitter_archive.join(json_df, on="tweet_id", how='outer',sort=False)
fullData = pd.concat([combinedData, imagePrediction])